In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
import tensorflow as tf

In [6]:
from tensorflow import keras

In [7]:
from keras import layers, models

In [8]:
x_train = train.drop('label', axis=1).iloc[0:31500]/255
y_train = train["label"][0:31500]

In [9]:
x_test = train.drop('label',axis=1).iloc[31500:]/255.0
y_test = train["label"][31500:]

In [10]:
x_train = np.array(x_train).reshape(31500,28,28,1)
x_test = np.array(x_test).reshape(10500,28,28,1)

In [11]:
model = models.Sequential()
model.add(layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28, 28,1))) # (28,28,3)
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(56, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(56, (3, 3), activation='relu'))

In [12]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 28)        280       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 28)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 56)        14168     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 56)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 56)          28280     
_________________________________________________________________
flatten (Flatten)            (None, 504)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [14]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [15]:
trained_df = model.fit(x_train, y_train, validation_data=(x_test, y_test))

985/985 [==============================] - 26s 25ms/step - loss: 0.4999 - accuracy: 0.8437 - val_loss: 0.0787 - val_accuracy: 0.9740


In [16]:
test = pd.read_csv('test.csv')

In [17]:
pred = model.predict(np.array(test).reshape(28000,28,28,1))

In [29]:
output = []

for i in range(len(pred)):
    j = -1
    max_i = -1
    out = 0
    for num in pred[i]:
        j += 1
        if num > max_i:
            max_i = num
            out = j
            
    output.append(out)

In [30]:
outputDict = {"ImageId":np.arange(1,28001)}
submissions = pd.DataFrame(outputDict)
submissions["Label"] = output
submissions.to_csv("submission.csv", index=False, header=True)